In [2]:
import pandas as pd 
import numpy as np

# State-Level controls

In [55]:
path = "/Users/jpmvbastos/Documents/GitHub/COVIDBR/Data/"
df_list = ['population','stategdp','health_insurance','hospitalbeds','icu','population65','doctors'] 
for i in df_list:
    x = pd.read_excel(path+i+".xls")
    x = pd.melt(x, id_vars=['Sigla','Codigo','Estado'], var_name='Year', value_name=i) # Use pd.melt() to convert to long format
    x['Year'] = x['Year'].astype('int64')
    x.sort_values(['Estado', 'Year'], inplace=True)
    globals()[i] = x    

In [58]:
df = stategdp
for i in [population, health_insurance, hospitalbeds, icu, population65, doctors]:
    df = pd.merge(df, i, on=['Sigla','Codigo','Estado','Year'], how='left')
df.Year = df.Year.astype(int)
df['Estado'] = df['Estado'].apply(lambda x: x.upper())
df['gdp_pc'] = df['stategdp']/df['population'] * 1000
df['health_insurance'] = df['health_insurance']/df['population'] * 100
df.rename(columns={'Estado':'State','Sigla':'UF','Codigo':'Code'}, inplace=True)
df

,UF,Code,State,Year,stategdp,population,health_insurance,hospitalbeds,icu,population65,doctors,gdp_pc
0,AC,12,ACRE,2000,4.840506e+06,541873.0,NaN,NaN,NaN,NaN,NaN,8932.915256
1,AC,12,ACRE,2001,5.024465e+06,574355.0,NaN,NaN,NaN,NaN,NaN,8748.013593
2,AC,12,ACRE,2002,5.644717e+06,586942.0,NaN,NaN,NaN,NaN,NaN,9617.162440
3,AC,12,ACRE,2003,5.623480e+06,600595.0,NaN,NaN,NaN,NaN,NaN,9363.181929
4,AC,12,ACRE,2004,5.847496e+06,630328.0,NaN,NaN,NaN,NaN,NaN,9276.909271
...,...,...,...,...,...,...,...,...,...,...,...,...
589,TO,17,TOCANTINS,2017,2.087139e+07,1550194.0,NaN,17.136089,2.221897,122.0,11.813619,13463.726442
590,TO,17,TOCANTINS,2018,2.088626e+07,1555229.0,NaN,18.103979,2.292267,137.0,12.151480,13429.698554
591,TO,17,TOCANTINS,2019,2.211289e+07,1572866.0,8.019564,19.356385,2.342857,132.0,12.166220,14058.977552
592,TO,17,TOCANTINS,2020,2.303456e+07,1590248.0,NaN,18.999657,3.312376,141.0,12.883735,14484.887991


In [218]:
df['population65'] = (df['population65']*1000)/df['population'] * 100

# Covid Cases

In [71]:
cases = pd.DataFrame()

for year in [2020,2021,2022]: 
    c = pd.read_csv(f'Data/Cases{year}.csv')
    c = c.groupby(['UF','Ano_Semana'])[['CasosAcumulados','ObitosAcumulados','CasosNovos','ObitosNovos']].sum().reset_index()
    cases = pd.concat([cases, c])

# Use the Ano_Semana column to extract the year and week number
cases['Year'] = cases['Ano_Semana'].str.split('/').str[1]
cases['Week'] = cases['Ano_Semana'].str.split('/').str[0]
cases.drop('Ano_Semana', axis=1, inplace=True)

cases = cases.set_index(['UF','Year','Week']).reset_index().sort_values(['UF','Year','Week'], ascending=[True,True,True])
cases.head(10)

,UF,Year,Week,CasosAcumulados,ObitosAcumulados,CasosNovos,ObitosNovos
0,AC,2020,13,25,0,25,0
1,AC,2020,14,46,0,11,0
2,AC,2020,15,72,3,9,3
3,AC,2020,16,142,6,27,3
4,AC,2020,17,258,11,34,3
5,AC,2020,18,556,22,96,4
6,AC,2020,19,1336,39,196,7
7,AC,2020,20,1872,59,202,7
8,AC,2020,21,3880,87,709,11
9,AC,2020,22,6072,142,646,19


In [162]:
# Merge Population data
pop = population[population['Year']==2019].rename(columns={'Sigla':'UF'})
pop = pop[['UF','population']]

cases = pd.merge(cases, pop, on=['UF'], how='left')

cases['cases_pc'] = cases['CasosAcumulados']/cases['population'] * 100000
cases['deaths_pc'] = cases['ObitosAcumulados']/cases['population'] * 100000

cases

,UF,Year,Week,CasosAcumulados,ObitosAcumulados,CasosNovos,ObitosNovos,population
0,AC,2020,13,25,0,25,0,881935
1,AC,2020,14,46,0,11,0,881935
2,AC,2020,15,72,3,9,3,881935
3,AC,2020,16,142,6,27,3,881935
4,AC,2020,17,258,11,34,3,881935
...,...,...,...,...,...,...,...,...
3937,TO,2022,52,359569,4224,2529,2,1572866
3938,TO,2022,6,289180,4046,2901,21,1572866
3939,TO,2022,7,294850,4086,1951,21,1572866
3940,TO,2022,8,298390,4106,1398,13,1572866


# Lockdown Stringency

### Simplified Yearly Average Scores

In [49]:
# Import Lockdown data
ld = pd.read_excel('Data/Lockdown Stringency.xlsx')

#Keep only the first three columns
ld = ld.iloc[:,0:3]

#Rename columns
ld.rename(columns={'Lockdown Stringency Score':"LSS"}, inplace=True)
ld['State'] = ld['State'].str.upper()

#Remove spaces from the begining or end of values in State column
ld['State'] = ld['State'].str.strip()

#Remove spaces from the begining or end of values in State column
ld['State'] = ld['State'].str.strip()

# Include UF column
ld['UF'] = ld['State'].map(brazilian_states)

ld.head()

,State,Year,LSS,UF
0,ACRE,2020,2.461193,AC
1,ALAGOAS,2020,4.108177,AL
2,AMAPÁ,2020,3.426266,AP
3,AMAZONAS,2020,2.937432,AM
4,BAHIA,2020,3.516136,BA


## Detailed Raw Data

In [195]:
si = pd.read_csv('Data/OxCGRT_raw_BRA_v1.csv', low_memory=False)
si = si[si['Jurisdiction'] == 'STATE_GOV']

si['RegionCode'] = si['RegionCode'].str[3:]
display(si)

,CountryName,CountryCode,RegionName,RegionCode,CityName,CityCode,Jurisdiction,Date,C1E_School.closing,C1E_Flag,...,StringencyIndex_WeightedAverage,GovernmentResponseIndex_NonVaccinated,GovernmentResponseIndex_Vaccinated,GovernmentResponseIndex_SimpleAverage,GovernmentResponseIndex_WeightedAverage,ContainmentHealthIndex_NonVaccinated,ContainmentHealthIndex_Vaccinated,ContainmentHealthIndex_SimpleAverage,ContainmentHealthIndex_WeightedAverage,EconomicSupportIndex
1096,Brazil,BRA,Acre,AC,State government,STATE_GOV,STATE_GOV,20200101,0.0,NaN,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
1097,Brazil,BRA,Acre,AC,State government,STATE_GOV,STATE_GOV,20200102,0.0,NaN,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
1098,Brazil,BRA,Acre,AC,State government,STATE_GOV,STATE_GOV,20200103,0.0,NaN,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
1099,Brazil,BRA,Acre,AC,State government,STATE_GOV,STATE_GOV,20200104,0.0,NaN,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
1100,Brazil,BRA,Acre,AC,State government,STATE_GOV,STATE_GOV,20200105,0.0,NaN,...,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58083,Brazil,BRA,Tocantins,TO,State government,STATE_GOV,STATE_GOV,20221227,0.0,NaN,...,11.11,12.5,12.5,12.5,12.5,14.29,14.29,14.29,14.29,0.0
58084,Brazil,BRA,Tocantins,TO,State government,STATE_GOV,STATE_GOV,20221228,0.0,NaN,...,11.11,12.5,12.5,12.5,12.5,14.29,14.29,14.29,14.29,0.0
58085,Brazil,BRA,Tocantins,TO,State government,STATE_GOV,STATE_GOV,20221229,0.0,NaN,...,11.11,12.5,12.5,12.5,12.5,14.29,14.29,14.29,14.29,0.0
58086,Brazil,BRA,Tocantins,TO,State government,STATE_GOV,STATE_GOV,20221230,0.0,NaN,...,11.11,12.5,12.5,12.5,12.5,14.29,14.29,14.29,14.29,0.0


In [103]:
# List of columns begginning with regex C+number+E
rest = si.filter(regex=r'^C\d+E', axis=1)
rest.columns

Index(['C1E_School.closing', 'C1E_Flag', 'C2E_Workplace.closing', 'C2E_Flag',
       'C3E_Cancel.public.events', 'C3E_Flag',
       'C4E_Restrictions.on.gatherings', 'C4E_Flag',
       'C5E_Close.public.transport', 'C5E_Flag',
       'C6E_Stay.at.home.requirements', 'C6E_Flag',
       'C7E_Restrictions.on.internal.movement', 'C7E_Flag',
       'C8E_International.travel.controls'],
      dtype='object')

In [193]:
# Restrictions vars 
vars = ['C1E_School.closing', 'C2E_Workplace.closing', 'C3E_Cancel.public.events', 
 'C4E_Restrictions.on.gatherings', 'C5E_Close.public.transport', 
 'C6E_Stay.at.home.requirements', 'C7E_Restrictions.on.internal.movement', 
 'C8E_International.travel.controls']

# For each RegionName, get the first at which any of the vars is 2
first = si.loc[si[vars].eq(2).any(axis=1), ['RegionName','RegionCode','Date','C1E_School.closing', 'C2E_Workplace.closing', 'C3E_Cancel.public.events'
                                    , 'C4E_Restrictions.on.gatherings', 'C5E_Close.public.transport', 'C6E_Stay.at.home.requirements', 
                                    'C7E_Restrictions.on.internal.movement', 'C8E_International.travel.controls','StringencyIndex_WeightedAverage']].groupby('RegionName').first()

first.reset_index(inplace=True)


first.rename(columns={'RegionName':'State'}, inplace=True)
first.rename(columns={'RegionCode':'UF','Date':'first_restriction','StringencyIndex_WeightedAverage':'first_stringency_index'}, inplace=True)
first = first[['UF','State','first_restriction','first_stringency_index']].sort_values('UF')

# Convert first_restriction to datetime
first['first_restriction'] = pd.to_datetime(first['first_restriction'], format='%Y%m%d')

# Get the week of the year from the first_restriction
first['week_first_restriction'] = first['first_restriction'].dt.isocalendar().week

first.head()

,UF,State,first_restriction,first_stringency_index,week_first_restriction
0,AC,Acre,2020-03-20,66.67,12
1,AL,Alagoas,2020-03-20,59.26,12
3,AM,Amazonas,2020-03-16,18.52,12
2,AP,Amapa,2020-03-18,29.63,12
4,BA,Bahia,2020-03-16,43.98,12


In [224]:
# Get the number of deaths on first week with available 
first['deaths_week13'] = cases[cases['CasosAcumulados'] > 1].groupby('UF').first().sort_values('UF')['ObitosAcumulados'].values
first['cases_week13'] = cases[cases['CasosAcumulados'] > 1].groupby('UF').first().sort_values('UF')['CasosAcumulados'].values

# If there no are deaths in week 1, get the first week with deaths
first['first_death'] = np.where(first['deaths_week13']==0, cases[cases['ObitosAcumulados'] > 0].groupby('UF').first().sort_values('UF')['Week'].values, 12)

# Get the first week where there are more than 1000 cases
first['first_1kcases'] = cases[cases['CasosAcumulados'] > 1000].groupby('UF').first().sort_values('UF')['Week'].values

#First week with more than X cases per capita
first['first_10cases_pc'] = cases[cases['cases_pc'] >= 10].groupby('UF').first().sort_values('UF')['Week'].values # 10 cases per 100k
first['first_100cases_pc'] = cases[cases['cases_pc'] >= 100].groupby('UF').first().sort_values('UF')['Week'].values # 100 cases per 100k

#First week with more than X death per capita
first['first_1death_pc'] = cases[cases['deaths_pc'] >= 1].groupby('UF').first().sort_values('UF')['Week'].values # 1 death per 100k
first['first_10death_pc'] = cases[cases['deaths_pc'] >= 10].groupby('UF').first().sort_values('UF')['Week'].values # 10 deaths per 100k

# Mean stringency index for 2020
first['si2020'] = si[si['Date'] < 20210101].groupby('RegionCode')['StringencyIndex_WeightedAverage'
                                                                  ].mean().reset_index().sort_values('RegionCode')['StringencyIndex_WeightedAverage'].values

first['si2021'] = si[(si['Date'] >= 20210101) & (si['Date'] < 20220101)].groupby('RegionCode')['StringencyIndex_WeightedAverage'
                                                                  ].mean().reset_index().sort_values('RegionCode')['StringencyIndex_WeightedAverage'].values

first.drop('State', axis=1, inplace=True)
first

,UF,first_restriction,first_stringency_index,week_first_restriction,deaths_week13,cases_week13,first_death,first_1kcases,first_10cases_pc,first_100cases_pc,first_1death_pc,first_10death_pc,si2020,si2021
0,AC,2020-03-20,66.67,12,0,25,15,19,16,19,17,22,54.816311,51.881836
1,AL,2020-03-20,59.26,12,0,14,14,18,17,20,18,22,50.393634,45.807123
3,AM,2020-03-16,18.52,12,1,111,12,15,15,18,15,18,33.297705,52.473425
2,AP,2020-03-18,29.63,12,0,4,14,18,15,18,16,20,54.508224,46.029342
4,BA,2020-03-16,43.98,12,0,128,14,16,17,22,19,26,47.094645,41.646959
5,CE,2020-03-16,30.56,12,4,322,12,15,15,19,16,19,51.775956,57.489589
6,DF,2020-03-14,43.52,11,0,260,14,17,14,20,18,25,37.158661,36.694000
7,ES,2020-03-18,33.33,12,0,53,14,16,16,19,17,21,50.806776,54.149945
8,GO,2020-03-13,27.78,11,1,56,12,19,18,24,21,28,47.038197,38.163397
9,MA,2020-03-16,22.22,12,0,14,14,16,16,20,17,21,38.390355,51.066466


# Economic Freedom Data

In [35]:
ef = pd.read_excel('Data/Serie_historica_2003-2019_subdimensoes_do_IMLEE.xlsx')
ef = pd.melt(ef, id_vars=['ID','UF'], var_name='Year', value_name='IMLEE').sort_values(['UF','Year'])

# Rename UF as State and replace its values 
ef.rename(columns={'UF':'State','ID':'Code'}, inplace=True)

brazilian_states = {
    'ACRE': 'AC',
    'ALAGOAS': 'AL',
    'AMAPÁ': 'AP',
    'AMAZONAS': 'AM',
    'BAHIA': 'BA',
    'CEARÁ': 'CE',
    'DISTRITO FEDERAL': 'DF',
    'ESPÍRITO SANTO': 'ES',
    'GOIÁS': 'GO',
    'MARANHÃO': 'MA',
    'MATO GROSSO': 'MT',
    'MATO GROSSO DO SUL': 'MS',
    'MINAS GERAIS': 'MG',
    'PARÁ': 'PA',
    'PARAÍBA': 'PB',
    'PARANÁ': 'PR',
    'PERNAMBUCO': 'PE',
    'PIAUÍ': 'PI',
    'RIO DE JANEIRO': 'RJ',
    'RIO GRANDE DO NORTE': 'RN',
    'RIO GRANDE DO SUL': 'RS',
    'RONDÔNIA': 'RO',
    'RORAIMA': 'RR',
    'SANTA CATARINA': 'SC',
    'SÃO PAULO': 'SP',
    'SERGIPE': 'SE',
    'TOCANTINS': 'TO'
}

ef['UF'] = ef['State'].map(brazilian_states)

#Keep only if year=2019
ef = ef[ef['Year'] >= 2018]

ef

,Code,State,Year,IMLEE,UF
390,12,ACRE,2018,7.15,AC
416,12,ACRE,2019,5.96,AC
442,12,ACRE,2020,3.11,AC
468,12,ACRE,2021,3.94,AC
391,27,ALAGOAS,2018,7.86,AL
...,...,...,...,...,...
491,35,SÃO PAULO,2021,6.03,SP
415,17,TOCANTINS,2018,7.55,TO
441,17,TOCANTINS,2019,4.93,TO
467,17,TOCANTINS,2020,3.16,TO


# Election Data

In [89]:
tse = pd.read_csv('Data/votacao_secao_2018_BR.csv', encoding='latin1', delimiter=";", low_memory=False)

In [96]:
# Presidential election
pr = tse.loc[(tse['DS_CARGO'] == 'PRESIDENTE') & (tse['NR_TURNO'] == 2) & (tse['SG_UF'] != 'ZZ')]

# Keep only the columns we need
pr = pr[['SG_UF','DS_CARGO','NM_VOTAVEL','QT_VOTOS']]
pr = pr.groupby(['SG_UF','DS_CARGO','NM_VOTAVEL']).sum().reset_index()
pr = pr.pivot_table(index=['SG_UF'], columns='NM_VOTAVEL', values='QT_VOTOS').reset_index()
pr.rename(columns={'SG_UF':'UF'}, inplace=True)

In [209]:
pr.rename(columns={'JAIR MESSIAS BOLSONARO':'votes_right',
                   'FERNANDO HADDAD':'votes_left'}, inplace=True)
pr['votes_null'] = pr['VOTO NULO']+pr['VOTO BRANCO']
pr['share_votes_right'] = pr['votes_right']/(pr['votes_right']+pr['votes_left']+pr['votes_null'])
pr.drop(['VOTO BRANCO','VOTO NULO'], axis=1, inplace=True)
pr

NM_VOTAVEL,UF,votes_left,votes_right,votes_null,share_votes_right
0,AC,86977,294899,17253,0.738856
1,AL,912034,610093,122332,0.370999
2,AM,875845,885401,148033,0.463736
3,AP,183616,185096,26227,0.468670
4,BA,5484901,2060382,655044,0.251256
5,CE,3407526,1384591,366226,0.268418
6,DF,463340,1080411,147645,0.638769
7,ES,747768,1276611,154116,0.586006
8,GO,1118060,2124739,249025,0.608490
9,MA,2428913,886565,162039,0.254942


# Merge

In [226]:
# Merge datasets
data = pd.merge(df, ef, on=['UF','Year','State','Code'], how='left')
data = pd.merge(data, ld, on=['UF','Year','State'], how='left').sort_values(['UF','Year'])
data = pd.merge(data, first, on=['UF'], how='left')
data = pd.merge(data, pr, on=['UF'], how='left')

data[data['Year'] >=2019].head()

,UF,Code,State,Year,stategdp,population,health_insurance,hospitalbeds,icu,population65,...,first_10cases_pc,first_100cases_pc,first_1death_pc,first_10death_pc,si2020,si2021,votes_left,votes_right,votes_null,share_votes_right
19,AC,12,ACRE,2019,8.782024e+06,881935.0,1.758633,16.654667,1.859547,6.122900,...,16,19,17,22,54.816311,51.881836,86977,294899,17253,0.738856
20,AC,12,ACRE,2020,8.694794e+06,894470.0,NaN,17.613782,3.022274,6.484287,...,16,19,17,22,54.816311,51.881836,86977,294899,17253,0.738856
21,AC,12,ACRE,2021,9.977504e+06,906876.0,NaN,17.926008,4.493448,5.513433,...,16,19,17,22,54.816311,51.881836,86977,294899,17253,0.738856
41,AL,27,ALAGOAS,2019,3.312990e+07,3337357.0,8.267890,17.171570,2.078769,9.049077,...,17,20,18,22,50.393634,45.807123,912034,610093,122332,0.370999
42,AL,27,ALAGOAS,2020,3.335269e+07,3351543.0,NaN,19.098501,2.765516,9.010775,...,17,20,18,22,50.393634,45.807123,912034,610093,122332,0.370999


In [236]:
cases['Year'] = cases['Year'].astype(int)
cases['Week'] = cases['Week'].astype(int)



#Get the total deaths in 2021
cases[cases['Year'] == 2021].groupby('UF')['ObitosAcumulados'].max()



UF
AC      795
AL     2489
AM     5285
AP      925
BA     9129
CE    10009
DF     4259
ES     5080
GO     6807
MA     4500
MG    11903
MS     2329
MT     4455
PA     7188
PB     3672
PE     9654
PI     2837
PR     7974
RJ    25530
RN     2993
RO     1818
RR      781
RS     8872
SC     5255
SE     2484
SP    46723
TO     1234
Name: ObitosAcumulados, dtype: int64

In [238]:
#Get the total deaths in 2020 
data.loc[data['Year']==2020, 'deaths2020'] = cases[cases['Year'] == 2020].groupby('UF')['ObitosAcumulados'].max().values
data.loc[data['Year']==2021, 'deaths2021'] = cases[cases['Year'] == 2021].groupby('UF')['ObitosAcumulados'].max().values

#Get the total cases in 2020
data.loc[data['Year']==2020, 'cases2020'] = cases[cases['Year'] == 2020].groupby('UF')['CasosAcumulados'].max().values
data.loc[data['Year']==2021, 'cases2021'] = cases[cases['Year'] == 2021].groupby('UF')['CasosAcumulados'].max().values

# Get the total cases per capita in 2020
data.loc[data['Year']==2020, 'cases_pc2020'] = cases[cases['Year'] == 2020].groupby('UF')['cases_pc'].max().values
data.loc[data['Year']==2021, 'cases_pc2021'] = cases[cases['Year'] == 2021].groupby('UF')['cases_pc'].max().values

# Get the total deaths per capita in 2020
data.loc[data['Year']==2020, 'deaths_pc2020'] = cases[cases['Year'] == 2020].groupby('UF')['deaths_pc'].max().values
data.loc[data['Year']==2021, 'deaths_pc2021'] = cases[cases['Year'] == 2021].groupby('UF')['deaths_pc'].max().values

In [239]:
data.to_csv('Data/COVIDBR.csv', index=False)